In [ ]:
{
    "cells": [
        {
            "cell_type": "markdown",
            "id": "9ac12f8f-86a9-433e-860a-3981f858ebd1",
            "metadata": {},
            "source": [
                "## **1. TLC 승차 건 카운팅**"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "2daf60a1",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "from pyspark import SparkConf, SparkContext\n",
                "import pandas as pd"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "806cced7",
            "metadata": {},
            "source": [
                "**SparkConf**\n",
                "사용자가 재정의해서 쓸 수 있는 설정 옵션들에 대한 키와 값을 갖고있는 객체\n",
                "\n",
                "\n",
                "**SparkContext**\n",
                "Spark 클러스터와 연결시켜주는 객체\n",
                "\n",
                "\n",
                "* Spark 모든 기능에 접근할 수 있는 시작점\n",
                "* Spark는 분산환경에서 동작하기 때문에 Driver Program 을 구동시키기 위해 SparkContext가 필요\n",
                "* SparkContext는 프로그램당 하나만 만들 수 있고 사용후에는 종료\n"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "2a7753cb",
            "metadata": {},
            "source": [
                "#### **SparkContext 초기화**\n",
                "\n",
                "![sparkcontext](assets/sparkcontext.png)\n",
                "\n",
                "* SparkContext 객체는 내부에 자바로 동작하는 Py4J의 SparkContext와 연결\n",
                "* 이 덕분에 파이썬으로 코딩하면서도 자바 위에서 동작하는 프로그램을 작성할 수 있다 \n",
                "* RDD를 만들 수 있다"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "acaae61e",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "# Configure Spark \n",
                "conf = SparkConf().setMaster(\"local\").setAppName(\"uber-date-trips\")\n",
                "sc = SparkContext(conf=conf).getOrCreate()"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "0548e00c",
            "metadata": {},
            "source": [
                "setMaster(\"local\") - 분산된 환경이 아닌 개발용 로컬 환경을 쓴다\n",
                "\n",
                "setAppName - Spark UI에서 확인 가능한 스파크 앱 이름"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "e183d6b8",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "# Dataset sample\n",
                "path = \"../data/\"  # data 디렉토리 경로\n",
                "filename = \"fhvhv_tripdata_2020-03_short.csv\""
            ]
        },
        {
            "cell_type": "markdown",
            "id": "4ee82272",
            "metadata": {},
            "source": [
                "#### **데이터 로딩후 RDD 생성**"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "39b8e403-2665-48c6-9fe8-01678a206ee5",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "lines = sc.textFile(path + filename)   # .csv -> RDD object"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "7772a2dd",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "header = lines.first() "
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "3eb2a3e8-387f-491e-9342-66b36e0149eb",
            "metadata": {},
            "outputs": [],
            "source": [
                "header"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "f277156a",
            "metadata": {},
            "source": [
                "*데이터*\n",
                "```\n",
                "hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag\n",
                "HV0005,B02510,2020-03-01 00:03:40,2020-03-01 00:23:39,81,159,\n",
                "HV0005,B02510,2020-03-01 00:28:05,2020-03-01 00:38:57,168,119,\n",
                "HV0003,B02764,2020-03-01 00:03:07,2020-03-01 00:15:04,137,209,1\n",
                "HV0003,B02764,2020-03-01 00:18:42,2020-03-01 00:38:42,209,80,\n",
                "HV0003,B02764,2020-03-01 00:44:24,2020-03-01 00:58:44,256,226,\n",
                "...\n",
                "```"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "5e5e7ad9",
            "metadata": {},
            "source": [
                "#### **필요한 부분만 추출하기**"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "2997f8d7",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "filtered_lines = lines.filter(lambda row:row != header) # all lines excepting the header"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "00690052",
            "metadata": {},
            "source": [
                "아래와 같습니다\n",
                "```python\n",
                "def f(row):\n",
                "    return row != header\n",
                "lines.filter(f) \n",
                "```\n",
                "\n",
                "*데이터*\n",
                "```\n",
                "HV0005,B02510,2020-03-01 00:03:40,2020-03-01 00:23:39,81,159, \n",
                "HV0005,B02510,2020-03-01 00:28:05,2020-03-01 00:38:57,168,119,\n",
                "HV0003,B02764,2020-03-01 00:03:07,2020-03-01 00:15:04,137,209,1\n",
                "HV0003,B02764,2020-03-01 00:18:42,2020-03-01 00:38:42,209,80,\n",
                "HV0003,B02764,2020-03-01 00:44:24,2020-03-01 00:58:44,256,226,\n",
                "...\n",
                "```"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "d6d5681a",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "dates = filtered_lines.map(lambda x: x.split(\",\")[2].split(\" \")[0])"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "a0be738a",
            "metadata": {},
            "source": [
                "```map()```함수로 우리가 원하는 부분만 추출 할 수 있다\n",
                "\n",
                "\n",
                "추출하는 함수\n",
                "```python\n",
                "lambda x: x.split(\",\")[2].split(\" \")[0]\n",
                "\n",
                "```\n",
                "아래와 같다\n",
                "\n",
                "```python]\n",
                "def f(x):\n",
                "    return x.split(\",\")[2].split(\" \")[0]\n",
                "```\n",
                "\n",
                "\n",
                "오리지널 데이터\n",
                "```\n",
                "HV0005,B02510,2020-03-01 00:03:40,2020-03-01 00:23:39,81,159,\n",
                "```\n",
                "\n",
                "x.split(\",\")\n",
                "```\n",
                "[HV0005,B02510,2020-03-01 00:03:40,2020-03-01 00:23:39,81,159,]\n",
                "```\n",
                "\n",
                "\n",
                "x.split(\",\")[2]\n",
                "```\n",
                "[2020-03-01 00:03:40]\n",
                "```\n",
                "\n",
                "\n",
                "x.split(\",\")[2].split(\" \")\n",
                "```\n",
                "[2020-03-01,00:03:40]\n",
                "```\n",
                "\n",
                "x.split(\",\")[2].split(\" \")[0]\n",
                "```\n",
                "2020-03-01\n",
                "```"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "118b51e7",
            "metadata": {},
            "source": [
                "#### **CountByValue( )**"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "befafcb4",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "result = dates.countByValue() # Transformation, Action"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "91841c20",
            "metadata": {},
            "source": [
                "값이 얼마나 등장하는지 세준다 \n",
                "\n",
                "예)\n",
                "```\n",
                "2020-03-01\n",
                "2020-03-01\n",
                "2020-03-01\n",
                "2020-03-02\n",
                "2020-03-02\n",
                "2020-03-03\n",
                "```\n",
                "countByValue()\n",
                "```\n",
                "(2020-03-01,3)\n",
                "(2020-03-02,2)\n",
                "(2020-03-03,1)\n",
                "```\n",
                "\n",
                "\n",
                "**result는 이제 더이상 RDD가 아닌 Python 객체**"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "fec310a3-26d6-4086-a9d9-cec5f9801a76",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "result"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "ac0de56a",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "# Save results as a csv file \n",
                "pd.Series(result, name=\"trips\").to_csv(\"trips_date.csv\")"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "fe8f9c44",
            "metadata": {
                "tags": []
            },
            "outputs": [],
            "source": [
                "# Visualize the results\n",
                "import matplotlib.pyplot as plt\n",
                "%matplotlib inline  \n",
                "\n",
                "trips = pd.read_csv(\"trips_date.csv\")\n",
                "trips.plot()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "id": "147348c7-5816-459e-af5e-81f2eb0ecc1d",
            "metadata": {},
            "outputs": [],
            "source": []
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3 (ipykernel)",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "codemirror_mode": {
                "name": "ipython",
                "version": 3
            },
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.12.10"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 5
}
